In [1]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torch import get_file_path
from torch.utils.data import Dataset, DataLoader
import os

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(0))

Tesla V100-SXM3-32GB


In [3]:
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_dir)

    def __getitem__(self, idx):
        img_path = self.img_dir[idx]
        image = Image.open(img_path)

        label = int(os.path.basename(img_path).split('_')[0])

        if self.transform:
            image = self.transform(image)

        return image, label

In [4]:
def get_file_paths(folder):
    return [os.path.join(folder, fname) for fname in os.listdir(folder) if fname.endswith('.jpg')]


train_data = get_file_paths('dataset/training')
val_data = get_file_paths('dataset/validation')
eval_data = get_file_paths('dataset/evaluation')

In [5]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [6]:
train_dataset = CustomImageDataset(train_data, transform=transform)
val_dataset = CustomImageDataset(val_data, transform=transform)
eval_dataset = CustomImageDataset(eval_data, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=32, shuffle=True)

In [7]:
image, label = train_dataset[0]

In [8]:
image.size()

torch.Size([3, 32, 32])

In [9]:
class_names = ['Bread', 'Dairy product', 'Dessert', 'Egg', 'Fried food', 'Meat', 'Noodles/Pasta', 'Rice', 'Seafood', 'Soup', 'Vegetable/Fruit']

In [10]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 12, 5)  # (12, 28, 28)
        self.pool = nn.MaxPool2d(2, 2)  # (12, 14, 14)
        self.conv2 = nn.Conv2d(12, 24, 4)  # (24, 10, 10) -> (24, 5, 5) -> Flatten (24 * 5 * 5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 11)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [11]:
net = NeuralNet().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [12]:
for epoch in range(10):
    print(f'Training epoch {epoch}...')

    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Loss: {running_loss / len(train_loader):.4f}')

Training epoch 0...
Loss: 2.3566
Training epoch 1...
Loss: 2.2670
Training epoch 2...
Loss: 2.2361
Training epoch 3...
Loss: 2.2086
Training epoch 4...
Loss: 2.1882
Training epoch 5...
Loss: 2.1681
Training epoch 6...
Loss: 2.1425
Training epoch 7...
Loss: 2.1108
Training epoch 8...
Loss: 2.0867
Training epoch 9...
Loss: 2.0651


In [13]:
torch.save(net.state_dict(), 'trained_nett.pth')

In [14]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_nett.pth'))

<All keys matched successfully>

In [15]:
correct = 0
total = 0

net.eval()
with torch.no_grad():
    for data in eval_loader:
        images, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total

print(f'Accuracy: {accuracy}%')

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!